# Weak-lensing galaxy shape catalogue validation

## Local metacalibration

Contents.
- Metacalibration (local)

> **_NOTE:_** Before running this notebook, set kernel to `main_set.ipynb'

In [ ]:
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from lenspack.geometry.projections.gnom import radec2xy
from lenspack.utils import bin2d

from sp_validation.io import *
from sp_validation.galaxy import *

In [ ]:
plot_dir_local = {}
for sh in shapes:
    plot_dir_local[sh] = f'{plot_dir}/local_cal_{sh}'

#### Enlarge field size

In [ ]:
# Calibration pixel sizes for local calibration, in degree
cal_pix_size_deg = [1, 2, 4]

# The following arrays need to have a length equal or larger n_cal
colors = ['blue', 'red', 'green']
linestyles = ['-', '-', '-']

n_cal = len(cal_pix_size_deg)

In [ ]:
# Compute larger field size, divisible by largest subfield size
factor = max(cal_pix_size_deg)

size_x_deg_new = {}
size_y_deg_new = {}

for sh in shapes:
    size_x_deg_new[sh] = np.ceil(size_x_deg[sh] / factor) * factor
    size_y_deg_new[sh] = np.ceil(size_y_deg[sh] / factor) * factor

    print(
        f'{sh}: Enlarged field size in projected coordinates is'
        + f' (x, y) = ({size_x_deg_new[sh]:.1f}, {size_y_deg_new[sh]:.1f}) deg')

In [ ]:
# Modify max coordinates to account for enlarged field

max_x_new = {}
max_y_new = {}
size_x_new = {}
size_y_new = {}

for sh in shapes:
    max_x_new[sh] = max_x[sh] + np.deg2rad(size_x_deg_new[sh] - size_x_deg[sh])
    max_y_new[sh] = max_y[sh] + np.deg2rad(size_y_deg_new[sh] - size_y_deg[sh])
    size_x_new[sh] = max_x_new[sh] - min_x[sh]
    size_y_new[sh] = max_y_new[sh] - min_y[sh]

    # Check that new size is ok
    print(
        f'{sh}: Check new field size:',
        np.rad2deg(size_x_new[sh]), np.rad2deg(size_y_new[sh])
    )

#### Local calibration

In [ ]:
def local_calib(
    dd,
    x,
    y,
    min_x,
    max_x,
    min_y,
    max_y,
    size_x_new,
    size_y_new,
    npix_x,
    npix_y,
    R_shear_moy,
    R_selec_moy,
    dm,
    sigm,
    stats_file=None,
    verbose=False
):
    """Local Calib
    
    Local calibration.
    """
    
    # Mean total calibration matrix
    R_tot_moy = R_shear_moy + R_selec_moy
    
    g1_final = np.array([])
    g2_final = np.array([])
    g1_final_dm = np.array([])
    g2_final_dm = np.array([])
    R_selec = np.zeros((2, 2, npix_y, npix_x))
    R_shear = np.zeros((2, 2, npix_y, npix_x))
    ra_ngmix2 = np.array([])
    dec_ngmix2 = np.array([])
    R_selec_std = np.zeros((2, 2, npix_y, npix_x))
    R_shear_std = np.zeros((2, 2, npix_y, npix_x))
    c1_ngmix_cut = np.zeros((npix_y, npix_x))
    c2_ngmix_cut = np.zeros((npix_y, npix_x))
    err_c1_ngmix_cut = np.zeros((npix_y, npix_x))
    err_c2_ngmix_cut = np.zeros((npix_y, npix_x))
    
    ngal = bin2d(x, y, npix=(npix_x, npix_y), extent=(min_x, max_x, min_y, max_y))

    # Cut the catalogue into calibration pixels -> dd_cut
    for i in range(npix_x):
        for j in range(npix_y):
            if i == npix_x and j != npix_y:
                dd_cut = dd[np.where((x >= min_x + (i*size_x_new/npix_x)) & (x <= max_x) \
                                     & (y >= min_y + (j*size_y_new/npix_y)) & (y < min_y + ((j+1)*size_y_new/npix_y)))]
            elif j== npix_x and i != npix_y:
                dd_cut = dd[np.where((x >= min_x + (i*size_x_new/npix_x)) & (x < min_x + ((i+1)*size_x_new/npix_x)) \
                                     & (y >= min_y + (j*size_y_new/npix_y)) & (y <= max_y))] 
            elif j == npix_x and i == npix_y:
                dd_cut = dd[np.where((x >= min_x + (i*size_x_new/npix_x)) & (x <= max_x) \
                                     & (y >= min_y + (j*size_y_new/npix_y)) & (y <= max_y ))]
            else:
                dd_cut = dd[np.where((x >= min_x + (i*size_x_new/npix_x)) & (x < min_x + ((i+1)*size_x_new/npix_x)) \
                                     & (y >= min_y + (j*size_y_new/npix_y)) & (y < min_y + ((j+1)*size_y_new/npix_y)))]
        
            # Mask of dd_cut
            cut_common = classification_galaxy_base(dd_cut)
            m_gal_ngmix_cut = classification_galaxy_ngmix(dd_cut, cut_common, stats_file=stats_file, verbose=verbose)

            # Apply metacal to dd_cut. No verbose output
            gal_metacal_ngmix_cut = metacal(dd_cut, m_gal_ngmix_cut, verbose=False)
        
            # Save ra & dec to keep the order        
            ra_ngmix_temp = dd_cut['XWIN_WORLD'][m_gal_ngmix_cut][gal_metacal_ngmix_cut.mask_dict['ns']]
            ra_ngmix2 = np.concatenate((ra_ngmix2, ra_ngmix_temp))
            dec_ngmix_temp = dd_cut['YWIN_WORLD'][m_gal_ngmix_cut][gal_metacal_ngmix_cut.mask_dict['ns']]
            dec_ngmix2 = np.concatenate((dec_ngmix2, dec_ngmix_temp))
            
            w_ngmix_cut = gal_metacal_ngmix_cut.ns['w'][gal_metacal_ngmix_cut.mask_dict['ns']]
        
            g_ngmix_cut = np.array([gal_metacal_ngmix_cut.ns['g1'][gal_metacal_ngmix_cut.mask_dict['ns']], gal_metacal_ngmix_cut.ns['g2'][gal_metacal_ngmix_cut.mask_dict['ns']]])
        
            # If numver of galaxy is ok, metacal local
            if ngal[j,i] > np.mean(ngal) / 2:
                g_corr_ngmix_cut = np.linalg.inv(gal_metacal_ngmix_cut.R).dot(g_ngmix_cut)
                
                # Add of delta m 
                R_dm = gal_metacal_ngmix_cut.R + np.ones((2,2)) * (dm + np.random.normal(0, sigm))
                g_corr_ngmix_cut_dm = np.linalg.inv(R_dm).dot(g_ngmix_cut)
                
                # Additive bias
                c1_ngmix_cut[j,i], err_c1_ngmix_cut[j,i] = jackknif_weighted_average2(g_corr_ngmix_cut[0], w_ngmix_cut, remove_size=0.05, n_realization=500)
                c2_ngmix_cut[j,i], err_c2_ngmix_cut[j,i] = jackknif_weighted_average2(g_corr_ngmix_cut[1], w_ngmix_cut, remove_size=0.05, n_realization=500)
                
                # Save R matrix and std
                R_selec[:,:,j,i] = gal_metacal_ngmix_cut.R_selection
                R_shear[:,:,j,i] = np.mean(gal_metacal_ngmix_cut.R_shear,2)
                                
            # If low number of galaxy, we use value of globaal metacal
            else:
                g_corr_ngmix_cut = np.linalg.inv(R_tot_moy).dot(g_ngmix_cut)
                R_dm = R_tot_moy + np.ones((2,2)) * (dm + np.random.normal(0, sigm))
                g_corr_ngmix_cut_dm = np.linalg.inv(R_dm).dot(g_ngmix_cut)
                
                c1_ngmix_cut[j,i] = 0
                err_c1_ngmix_cut[j,i] = 0
                c2_ngmix_cut[j,i] = 0
                err_c2_ngmix_cut[j,i] = 0
            
                R_selec[:,:,j,i] = R_selec_moy
                R_shear[:,:,j,i] = R_shear_moy
                             
            g1_final = np.concatenate((g1_final, g_corr_ngmix_cut[0]))  
            g2_final = np.concatenate((g2_final, g_corr_ngmix_cut[1]))
            g1_final_dm = np.concatenate((g1_final_dm, g_corr_ngmix_cut_dm[0]))  
            g2_final_dm = np.concatenate((g2_final_dm, g_corr_ngmix_cut_dm[1]))
            
            g_final = np.array([g1_final, g2_final])
            g_final_dm = np.array([g1_final_dm, g2_final_dm])
            c_ngmix_cut = np.array([c1_ngmix_cut, c2_ngmix_cut])
            err_c_ngmix_cut = np.array([err_c1_ngmix_cut, err_c2_ngmix_cut])
    
    return g_final, g_final_dm, R_shear, R_selec, ra_ngmix2, dec_ngmix2, c_ngmix_cut, err_c_ngmix_cut

In [ ]:
g_corr_local = {}
g_corr_local_dm = {}
R_shear_local = {}
R_selec_local = {}
ra_local = {}
dec_local = {}
c_local = {}
c_err_local = {}

# Additional mean and std of multiplicative bias
m = 0
dm = 0

# Loop over different calibration pixel sizes
for sh in shapes:
    g_corr_local[sh] = {}
    g_corr_local_dm[sh] = {}
    R_shear_local[sh] = {}
    R_selec_local[sh] = {}
    ra_local[sh] = {}
    dec_local[sh] = {}
    c_local[sh] = {}
    c_err_local[sh] = {}

    for cal_pix in cal_pix_size_deg:
        npix_x = int(size_x_deg_new[sh] / cal_pix)
        npix_y = int(size_y_deg_new[sh] / cal_pix)
        if verbose:
            print(
                f'{sh}: Calibration pixel size = {cal_pix} deg,'
                + f' number of pixels = {npix_x} x {npix_y}'
            )
    
        # Call local_calibration with verbose=False to avoid too much std output
        (
            g_corr_local[sh][cal_pix],
            g_corr_local_dm[sh][cal_pix],
            R_shear_local[sh][cal_pix],
            R_selec_local[sh][cal_pix],
            ra_local[sh][cal_pix],
            dec_local[sh][cal_pix],
            c_local[sh][cal_pix],
            c_err_local[sh][cal_pix]
        ) = local_calib(
            dd, 
            x[sh], 
            y[sh], 
            min_x[sh],
            max_x_new[sh],
            min_y[sh],
            max_y_new[sh],
            size_x_new[sh],
            size_y_new[sh],
            npix_x,
            npix_y,
            R_shear[sh],
            gal_metacal[sh].R_selection,
            m,
            dm,
            stats_file=None,
            verbose=False
        )

## Get quantities calibrated for local multiplicative and global additive biase

In [ ]:
g_corr_local_m_global_c = {}

for sh in shapes:
    g_corr_local_m_global_c[sh] = {}
    
    for cal_pix in cal_pix_size_deg:
        g_corr_local_m_global_c[sh][cal_pix] = np.zeros_like(g_corr_local[sh][cal_pix])
        for comp in (0, 1):
            g_corr_local[sh][cal_pix][comp] = g_corr_local[sh][cal_pix][comp] - c[sh][comp]

### Plots

In [ ]:
def sub_plot(nx, ny, i, im, title, fontsize=32):
    """Sub Plot
    
    Create sub plot.
    
    Parameters
    ----------
    nx, ny : int
        number of rows and columns
    i : int
        subplot number
    im : 2D array of float
        image
    title : string
        plot title text
    fontsize : int, optional, default=32
        title font size
    """

    plt.subplot(nx, ny, i)
        
    ax = plt.gca()
    im = ax.imshow(im)
        
    plt.title(title ,fontsize=fontsize)
        
    divider = make_axes_locatable(ax)
        
    plt.xticks(fontsize=fontsize)

    plt.yticks(fontsize=fontsize)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    plt.colorbar(im, cax=cax)
    plt.yticks(fontsize=fontsize)

#### Additive bias

In [ ]:
# Spatial variations
fontsize = 32

for sh in shapes:
    plt.figure(figsize=(74, 45))

    for i, cal_pix in enumerate(cal_pix_size_deg):

        for comp in (0, 1):
            subf = 4*i + 2*comp + 1
        
            sub_plot(
                n_cal,
                4, 
                subf,
                c_local[sh][cal_pix][comp],
                rf'{sh}: $c_{comp+1}$'
            )
            sub_plot(
                n_cal,
                4,
                subf+1,
                c_err_local[sh][cal_pix][comp],
                rf'{sh}: $\sigma(c_{comp+1})$'
            )

    plt.savefig(
        f'{plot_dir_local[sh]}/c_spatial.pdf',
        bbox_inches='tight',
        pad_inches=0
    )

### Response matrices

In [ ]:
# Uncertainty via jackknife

def compute_R_std(npix_x, npix_y, factor, R):
    """Compute R Std
    """
    
    R_std = np.zeros((2, 2, npix_y, npix_x))

    for i in range(npix_y):
        for j in range(npix_x):
            
            for c1 in (0, 1):
                for c2 in (0, 1):
                    R_ref = R[
                        c1,c2,factor*i:factor*(i+1),
                        factor*j:factor*(j+1)
                    ]
                    if np.sum(R_ref) != 0:
                        R_ref_1d = np.ravel(R_ref)
                        R_std[c1, c2, i, j] = jackknif_weighted_average(
                            R_ref_1d,
                            np.ones_like(R_ref_1d)
                        )[1]
                    else:
                        R_std[c1,c2,i,j] = np.nan
                
    return R_std

In [ ]:
R_shear_std_local = {}
R_selec_std_local = {}

# Reference = minimum pixel size
cal_pix_ref = min(cal_pix_size_deg)

for sh in shapes:
    R_shear_std_local[sh] = {}
    R_selec_std_local[sh] = {}

    for cal_pix in cal_pix_size_deg:

        if cal_pix == cal_pix_ref:
            continue
        
        npix_x = int(size_x_deg_new[sh] / cal_pix)
        npix_y = int(size_y_deg_new[sh] / cal_pix)
        if verbose:
            print(f'{sh}: Calibration pixel size = {cal_pix} deg')
    
        factor = int(cal_pix / cal_pix_ref)
    
        R_shear_std_local[sh][cal_pix] = compute_R_std(
            npix_x,
            npix_y,
            factor,
            R_shear_local[sh][cal_pix_ref]
        )
        R_selec_std_local[sh][cal_pix] = compute_R_std(
            npix_x,
            npix_y,
            factor,
            R_selec_local[sh][cal_pix_ref]
        )

In [ ]:
# Pixelise number of galaxies
ngal = {}

for sh in shapes:
    ngal[sh] = {}
    for cal_pix in cal_pix_size_deg:
    
        npix_x = int(size_x_deg_new[sh] / cal_pix)
        npix_y = int(size_y_deg_new[sh] / cal_pix)

        ngal[sh][cal_pix] = bin2d(
            x[sh],
            y[sh],
            npix=(npix_x, npix_y),
            extent=(
                min_x[sh],
                max_x_new[sh],
                min_y[sh],
                max_y_new[sh]
            )
        )

In [ ]:
# Plot shear matrix

for sh in shapes:
    plt.figure(figsize=(55, 45))

    c1 = 0
    c2 = 0

    for i, cal_pix in enumerate(cal_pix_size_deg):

        subf = 3*i + 1
        sub_plot(
            n_cal,
            3,
            subf,
            R_shear_local[sh][cal_pix][c1, c2],
            rf'{sh}: $R_{{\rm shear, {c1}, {c2}}}$'
        )
        if cal_pix in R_selec_std_local[sh]:
            sub_plot(
                n_cal,
                3,
                subf+1,
                R_shear_std_local[sh][cal_pix][0, 0],
                rf'{sh}: $\sigma(R_{{\rm shear, {c1}, {c2}}})$'
            )
        sub_plot(
            n_cal,
            3,
            subf+2,
            ngal[sh][cal_pix],
            rf'{sh}: $n_{{\rm gal}}$'
        )

    plt.savefig(f'{plot_dir_local[sh]}/R_shear.pdf')

In [ ]:
# Plot selection matrix

for sh in shapes:
    plt.figure(figsize=(55, 45))

    c1 = 0
    c2 = 0

    for i, cal_pix in enumerate(cal_pix_size_deg):

        subf = 3*i + 1
        sub_plot(
            n_cal,
            3,
            subf,
            R_selec_local[sh][cal_pix][c1, c2],
            rf'{sh}: $R_{{\rm selection, {c1}, {c2}}}$'
        )
        if cal_pix in R_selec_std_local[sh]:
            sub_plot(
                n_cal,
                3,
                subf+1,
                R_selec_std_local[sh][cal_pix][0, 0],
                rf'{sh}: $\sigma(R_{{\rm selection, {c1}, {c2}}})$'
            )
        sub_plot(
            n_cal, 
            3,
            subf+2,
            ngal[sh][cal_pix],
            rf'{sh}: $n_{{\rm gal}}$'
        )

    plt.savefig(f'{plot_dir_local[sh]}/R_selec.pdf')

In [ ]:
# Histograms

y_label = 'frequency'
n_bin = 50

In [ ]:
x_range = [-0.01, 0.01]

for sh in shapes:
    title = f'{sh}: Local additive bias'
    for comp in (0, 1):

        xs = []
        labels = []
        x_label = rf'$c_{comp+1}$'
        out_path = f'{plot_dir_local[sh]}/c_{comp+1}_hist.pdf'

        for cal_pix in cal_pix_size_deg:
        
            xs.append(np.ravel(c_local[sh][cal_pix][comp]))
            labels.append(f'{cal_pix} deg')

        plot_histograms(
            xs, 
            labels, 
            title,
            x_label,
            y_label,
            x_range,
            n_bin,
            out_path,
            colors=colors,
            linestyles=linestyles
        )

In [ ]:
x_range = [[0.5, 1], [-0.25, 0.25]]

for sh in shapes:
    title = f'{sh}: Local shear response matrix'
    for i, comp in enumerate(((0, 0), (0, 1))):

        xs = []
        labels = []
        x_label = rf'$R_{{\rm shear, {comp[0]}, {comp[1]}}}$'
        out_path = f'{plot_dir_local[sh]}/R_shear_hist.pdf'

        for cal_pix in cal_pix_size_deg:
        
            xs.append(np.ravel(R_shear_local[sh][cal_pix][comp[0], comp[1]]))
            labels.append(f'{cal_pix} deg')

        plot_histograms(
            xs,
            labels,
            title,
            x_label,
            y_label,
            x_range[i],
            n_bin,
            out_path,
            colors=colors,
            linestyles=linestyles
        )

In [ ]:
x_range = [[-0.25, 0.02], [-0.1, 0.1]]

for sh in shapes:
    title = f'{sh}: Local selection response matrix'
    for i, comp in enumerate(((0, 0), (0, 1))):

        xs = []
        labels = []
        x_label = rf'$R_{{\rm selection, {comp[0]}, {comp[1]}}}$'
        out_path = f'{plot_dir_local[sh]}/R_selec_hist.pdf'

        for cal_pix in cal_pix_size_deg:

            xs.append(np.ravel(R_selec_local[sh][cal_pix][comp[0], comp[1]]))
            labels.append(f'{cal_pix} deg')

        plot_histograms(
            xs,
            labels,
            title,
            x_label,
            y_label,
            x_range[i],
            n_bin,
            out_path,
            colors=colors,
            linestyles=linestyles
        )

## Get quantities locally calibrated for both multiplicative and additive biase

In [ ]:
### Extend local calibration quantities to 